In [41]:
# !pip install pyautogen
# !pip install openai

# or pip install -r requirements.txt (from the github repo)

Quick explanation for the retrieval-augmented generation example:

[source](https://microsoft.github.io/autogen/blog/2023/10/18/RetrieveChat#:~:text=Retrieval%20augmentation%20has,for%20the%20context.)

![](2024-07-08-10-20-10.png)

Retrieval augmentation has emerged as a practical and effective approach for mitigating the intrinsic limitations of LLMs by incorporating external documents. In this blog post, we introduce RAG agents of AutoGen that allows retrieval-augmented generation. The system consists of two agents: a Retrieval-augmented User Proxy agent, called RetrieveUserProxyAgent, and a Retrieval-augmented Assistant agent, called RetrieveAssistantAgent, both of which are extended from built-in agents from AutoGen. The overall architecture of the RAG agents is shown in the figure above.

To use Retrieval-augmented Chat, one needs to initialize two agents including Retrieval-augmented User Proxy and Retrieval-augmented Assistant. Initializing the Retrieval-Augmented User Proxy necessitates specifying a path to the document collection. Subsequently, the Retrieval-Augmented User Proxy can download the documents, segment them into chunks of a specific size, compute embeddings, and store them in a vector database. Once a chat is initiated, the agents collaboratively engage in code generation or question-answering adhering to the procedures outlined below:

The Retrieval-Augmented User Proxy retrieves document chunks based on the embedding similarity, and sends them along with the question to the Retrieval-Augmented Assistant.
The Retrieval-Augmented Assistant employs an LLM to generate code or text as answers based on the question and context provided. If the LLM is unable to produce a satisfactory response, it is instructed to reply with “Update Context” to the Retrieval-Augmented User Proxy.
If a response includes code blocks, the Retrieval-Augmented User Proxy executes the code and sends the output as feedback. If there are no code blocks or instructions to update the context, it terminates the conversation. Otherwise, it updates the context and forwards the question along with the new context to the Retrieval-Augmented Assistant. Note that if human input solicitation is enabled, individuals can proactively send any feedback, including Update Context”, to the Retrieval-Augmented Assistant.
If the Retrieval-Augmented Assistant receives “Update Context”, it requests the next most similar chunks of documents as new context from the Retrieval-Augmented User Proxy. Otherwise, it generates new code or text based on the feedback and chat history. If the LLM fails to generate an answer, it replies with “Update Context” again. This process can be repeated several times. The conversation terminates if no more documents are available for the context.

In [1]:
from autogen import config_list_from_json
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import os

/Users/greatmaster/miniconda3/envs/oreilly-autogen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config_list = config_list_from_json("./OAI_CONFIG_LIST")

In [3]:
assistant = RetrieveAssistantAgent(
      name="assistant",
      system_message="You are a helpful assistant.",
      llm_config={
          "timeout": 600,
          "cache_seed": 42,
          "config_list": config_list,
      },
  )

In [4]:
assistant

In [5]:

type(assistant)

autogen.agentchat.contrib.retrieve_assistant_agent.RetrieveAssistantAgent

In [6]:
ragproxyagent = RetrieveUserProxyAgent(
      name="ragproxyagent",
      human_input_mode="NEVER",
      max_consecutive_auto_reply=3,
      retrieve_config={
          "task": "code",
          "docs_path": [
              "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Examples/Integrate%20-%20Spark.md",
              "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md",
              os.path.join(os.path.abspath(""), "..", "website", "docs")],
      },
      code_execution_config=False,  # set to False if you don't want to execute the code
  )

In [48]:
assistant.reset()

In [8]:
code_problem = "How can I use FLAML to perform a classification task and use spark to do parallel training. Train 30 seconds and force cancel jobs if time limit is reached."
ragproxyagent.initiate_chat(assistant, problem=code_problem, search_string="spark")

ragproxyagent (to assistant):

How can I use FLAML to perform a classification task and use spark to do parallel training. Train 30 seconds and force cancel jobs if time limit is reached.

--------------------------------------------------------------------------------
assistant (to ragproxyagent):

FLAML (Fast Lightweight AutoML) is a Python library designed for efficient and effective AutoML, enabling automated machine learning. Integrating FLAML with Apache Spark for parallel training can speed up the process, especially when dealing with large datasets.

Here's a step-by-step guide to integrate FLAML with Spark for a classification task with a 30-second time limit:

### Step 1: Set up the environment
Ensure you have the necessary libraries installed. You need `pyspark` and `flaml` at the very least. You can install them using pip if they are not already installed:

```bash
pip install pyspark flaml
```

### Step 2: Import the necessary libraries
```python
from pyspark.sql import Sp

ChatResult(chat_id=None, chat_history=[{'content': 'How can I use FLAML to perform a classification task and use spark to do parallel training. Train 30 seconds and force cancel jobs if time limit is reached.', 'role': 'assistant'}, {'content': 'FLAML (Fast Lightweight AutoML) is a Python library designed for efficient and effective AutoML, enabling automated machine learning. Integrating FLAML with Apache Spark for parallel training can speed up the process, especially when dealing with large datasets.\n\nHere\'s a step-by-step guide to integrate FLAML with Spark for a classification task with a 30-second time limit:\n\n### Step 1: Set up the environment\nEnsure you have the necessary libraries installed. You need `pyspark` and `flaml` at the very least. You can install them using pip if they are not already installed:\n\n```bash\npip install pyspark flaml\n```\n\n### Step 2: Import the necessary libraries\n```python\nfrom pyspark.sql import SparkSession\nfrom flaml import AutoML\nfro